<a href="https://colab.research.google.com/github/Rsych/Machine-learning-with-tensorflow/blob/dev/Machine-learning-with-tensorflow/3_Tensorflow_CNN/2_Machine_learning_with_Tensorflow_DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning with Tensorflow - DCNN

In [13]:
# from google.colab import drive
# drive.mount('/content/drive')

In [14]:
import tensorflow as tf
from keras import optimizers, Sequential
from tensorflow.keras import datasets
from keras.layers import *
# from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout
import numpy as np

In [15]:
#@title Callbacks function define
%load_ext tensorboard
# %reload_ext tensorboard
import datetime
import os

# Create a function to build a TensorBoard callback
def create_tensorboard_callback():
    # Create a log directory for storing TensorBoard logs
    logdir = os.path.join('drive/MyDrive/logs/tf_cnn',
                          # Make it so the logs get tracked whenever we run an experiment
                          datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    return tf.keras.callbacks.TensorBoard(logdir)

def model_checkpoint():
    filepath = os.path.join('drive/MyDrive/models/model.{epoch:02d}-{val_loss:.2f}.h5')
    return tf.keras.callbacks.ModelCheckpoint(filepath,
                                              monitor='val_loss',
                                              save_best_only=True)

# Early stop callback
def early_stop():
    return tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5)

myCallbacks = [
               create_tensorboard_callback(),
               model_checkpoint(),
               early_stop()]


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
import scipy.stats as ss
# transform and load data
def norm_data(data):
    data_standadized_ss = ss.zscore(data)
    data_standadized_ss
    return data_standadized_ss

def tl_data():
    # load data
    (x_train, y_train), (x_test, y_test) = datasets.cifar100.load_data(label_mode="fine")
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # Normalization 
    x_train = norm_data(x_train)
    x_test = norm_data(x_test)

    y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

    return x_train, y_train, x_test, y_test

In [17]:
# Constant for model
BATCH_SIZE = 128
EPOCHS = 50
CLASSES = 100
VERBOSE = 1
VALIDATION_SPLIT = 0.2

In [18]:
def build_model():
    model = Sequential()
    
    # First block
    model.add(Conv2D(32, (3,3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.2))

    # Second block
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.3))

    # Third block
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.4))

    # Dense output
    model.add(Flatten())
    model.add(Dense(CLASSES, activation='softmax'))

    model.summary()
    
    return model

In [19]:
(x_train, y_train, x_test, y_test) = tl_data()
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)       

In [20]:
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
          callbacks=myCallbacks, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print(f'\nTest score: {score[0]}')
print(f'Test accuracy: {score[1]}')

Epoch 1/50
153/391 [==========>...................] - ETA: 7s - loss: 4.8102 - accuracy: 0.0609

KeyboardInterrupt: ignored

In [ ]:
x_train.shape[1:]

In [ ]:
%tensorboard --logdir drive/MyDrive/logs/tf_cnn

In [ ]:
import numpy as np
data = np.random.randint(30, size=(6, 5))
data

In [ ]:
mean = np.mean(data, axis=0)
std = np.std(data, axis=0)
np_norm = (data-mean)/(std+1e-7)
np_norm

In [ ]:
import scipy.stats as ss
data_standadized_ss = ss.zscore(data)
data_standadized_ss

In [ ]:
from sklearn.preprocessing import StandardScaler

data_standadized_skl = StandardScaler().fit_transform(data)

data_standadized_skl

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
# tf_norm_layer = preprocessing.Normalization()
# tf_norm_layer.adapt(data)
# tf_norm_data = tf_norm_layer(data)
# tf_norm_data
normalizer = preprocessing.Normalization()
normalizer.adapt(data)
print(data)

In [ ]:
norm_data = normalizer(data)
print(norm_data)